In [265]:
import matplotlib.pyplot as plt

import librosa
import numpy as np

def condition_label(wav_array, annot, sr, neg_type=1, condition=0.1):
    if neg_type == 1:
        neg_id = np.where(annot != 1)[0]
        pos_id = np.where(annot == 1)[0]

        base_signal = wav_array.copy()
        neg_signal = wav_array.copy()

        for n in neg_id:
            n_init = int(n * sr / 2)
            base_signal[n_init:int(n_init+sr/2)] = np.nan

        for p in pos_id:
            p_init = int(p * sr / 2)
            neg_signal[p_init:int(p_init+sr/2)] = np.nan

    elif neg_type == 2:
        abs_wav = np.abs(wav_array)
        neg_id = []
        
        ix = 0
        while (ix * 1000 + 999) < len(abs_wav):
            i = ix * 1000
            if (np.sum(abs_wav[i:i+1000]) <= condition):
                neg_id += list(range(i, i+1000))
            
            ix += 1
            
        pos_id = list(set(list(range(len(wav_array)))) - set(neg_id))
        
        base_signal = wav_array.copy()
        neg_signal = wav_array.copy()
        
        base_signal[neg_id] = np.nan
        neg_signal[pos_id] = np.nan
        
    else:
        abs_wav = np.abs(wav_array)
        neg_id = []
        
        ix = 0
        while (ix * 2000 + 1999) < len(abs_wav):
            i = ix * 2000
            if (np.sum(abs_wav[i:i+2000]) <= condition):
                if annot[int(ix / (sr /2))] != 0:
                    neg_id += list(range(i, i+2000))
            
            ix += 1
            
        pos_id = list(set(list(range(len(wav_array)))) - set(neg_id))
        
        base_signal = wav_array.copy()
        neg_signal = wav_array.copy()
        
        base_signal[neg_id] = np.nan
        neg_signal[pos_id] = np.nan
        
    return base_signal, neg_signal

In [ ]:
# (a) Audioset
file_name = './samples/audioset.mp3'
wav, sr = librosa.load(file_name)

init = sr * 16
wav = wav[init:(init + sr*10)]
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.array([0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

b, n = condition_label(wav, annot, sr)

# Plot daily and weekly resampled time series together
fig, axes = plt.subplots(2,3, figsize=(14,5))
ax = axes.flatten()[0]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(a) AudioSet []')


# (b) Chime-Home
file_name = './samples/chime_home.wav'
wav, sr = librosa.load(file_name)

t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.array([0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1])

b, n = condition_label(wav, annot, int(sr/2))

# Plot daily and weekly resampled time series together
ax = axes.flatten()[1]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(b) Chime-home []')

# (d) Libri-speech (5s)
file_name = './samples/librispeech.flac'
wav, sr = librosa.load(file_name)
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.array([0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1])

b, n = condition_label(wav, annot, sr, neg_type=2, condition=10)

# Plot daily and weekly resampled time series together
ax = axes.flatten()[3]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(d) Libri-Speech []')


# (e) ESC (5s)
file_name = './samples/ESC.wav'
wav, sr = librosa.load(file_name)
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

b, n = condition_label(wav, annot, sr, neg_type=1)

# Plot daily and weekly resampled time series together
ax = axes.flatten()[4]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(e) ESC []')

# (c) Battlesound-GUN (0.5s)
file_name = './samples/battlesound_gun.wav'
wav, sr = librosa.load(file_name)
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.load('./samples/battlesound_gun.npy')

b, n = condition_label(wav, annot, sr, neg_type=1)

# Plot daily and weekly resampled time series together
ax = axes.flatten()[2]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(c) BattleSound-GUN (Ours)')


# (f) Battlesound-VOICE (0.5s)
file_name = './samples/battlesound_voice.wav'
wav, sr = librosa.load(file_name)
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.load('./samples/battlesound_voice.npy')

b, n = condition_label(wav, annot, sr, neg_type=1)

# Plot daily and weekly resampled time series together
ax = axes.flatten()[5]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(f) BattleSound-VOICE (Ours)')

fig.set_facecolor('w')
plt.tight_layout() 
plt.savefig('Fig2-audio-imp.png', dpi=300)

/home/lilka/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [ ]:
# (a) Audioset
file_name = './samples/audioset.mp3'
wav, sr = librosa.load(file_name)

init = sr * 16
wav = wav[init:(init + sr*10)]
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.array([0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

b, n = condition_label(wav, annot, sr)

# Plot daily and weekly resampled time series together
fig, axes = plt.subplots(2,3, figsize=(14,5))
ax = axes.flatten()[0]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(a) AudioSet [9]')


# (b) Chime-Home
file_name = './samples/chime_home.wav'
wav, sr = librosa.load(file_name)

t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.array([0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1])

b, n = condition_label(wav, annot, int(sr/2))

# Plot daily and weekly resampled time series together
ax = axes.flatten()[1]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(b) Chime-home [12]')

# (c) Libri-speech (5s)
file_name = './samples/librispeech.flac'
wav, sr = librosa.load(file_name)
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.array([0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1])

b, n = condition_label(wav, annot, sr, neg_type=2, condition=10)

# Plot daily and weekly resampled time series together
ax = axes.flatten()[2]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(c) Libri-Speech [18]')


# (d) ESC (5s)
file_name = './samples/ESC.wav'
wav, sr = librosa.load(file_name)
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

b, n = condition_label(wav, annot, sr, neg_type=1)

# Plot daily and weekly resampled time series together
ax = axes.flatten()[3]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(d) ESC [16]')

# (e) Battlesound-GUN (0.5s)
file_name = './samples/battlesound_gun.wav'
wav, sr = librosa.load(file_name)
wav = wav[:8*sr]
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.load('./samples/battlesound_gun.npy')

b, n = condition_label(wav, annot, sr, neg_type=3, condition=10)

# Plot daily and weekly resampled time series together
ax = axes.flatten()[4]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(e) BattleSound-EVENT (Ours)')


# (f) Battlesound-VOICE (0.5s)
file_name = './samples/battlesound_voice.wav'
wav, sr = librosa.load(file_name)
t = np.array(list(range(0, len(wav)))) / sr # Time

annot= np.load('./samples/battlesound_voice.npy')
b, n = condition_label(wav, annot, sr, neg_type=3, condition=10)

# Plot daily and weekly resampled time series together
ax = axes.flatten()[5]
ax.plot(t, b, 'g', alpha=0.5)
ax.plot(t, n, 'r', alpha=0.5)
ax.grid(True, linewidth= 0.5, linestyle="--")
ax.set_title('(f) BattleSound-VOICE (Ours)')

fig.set_facecolor('w')
plt.tight_layout() 
plt.savefig('Fig2-audio-exampls.png', dpi=300)